### SQLAlchemy

In [2]:
import pandas as pd
import requests 
import time
from sqlalchemy import *
# import MySQLdb

> 크롤링하는 py파일에 sqlalchemy의 engine연결 후 df.to_sql() 시도\
**-> sql 내 테이블은 만들어졌으나 아래와 같은 에러메세지, 데이터 들어가지지않음**

In [5]:
import configparser

config = configparser.ConfigParser()
config.read("../db.ini")
sql_datas = config["sql"]

In [7]:
%%writefile baemin_del_sqlalchemy.py

import pandas as pd
import requests 
import time
from sqlalchemy import *
import configparser

config = configparser.ConfigParser()
config.read("../db.ini")
sql_datas = config["sql"]

# 카테고리 수정 전(참고용)
# categories = {
#     100104 : "플라스틱용기",
#     100105 : "종이용기/박스",
#     100138 : "카페/음료용기",
#     100159 : "종이/비닐봉투",
#     100165 : "수저/위생용품",
#     100177 : "스티커/배너/포스터",
#     100184 : "포장재",
#     100188 : "기타가게용품",
# }

# 카테고리 수정 후 
categories = {
    100104 : "포장용품",
    100105 : "포장용품",
    100138 : "포장용품",
    100159 : "포장용품",
    100165 : "가게위생용품",
    100184 : "포장용품",
    100188 : "가게위생용품",
}


def baemin_items(category, page):
    url = "https://mart.baemin.com/api/v2/front/categories/goods/{}/paging?goodsSortType=BASIC&page={}".format(category, page)
    response = requests.get(url)
    elements = response.json()['data']['simpleGoodsDtoPage']['content']
    datas = []
    for element in elements:
        title = element['name']
        price = element['goodsPrice']
#         o_price = element['customerPrice']
        # 원인은 모르겠지만.. 100050/밀가루 쪽에서 에러남
        try :
            min_gram = element['sizeDesc']
        except:
            min_gram = "-"
        try:
            price_per_gram = element['priceDesc']
        except:
            price_per_gram = "-"
        link = "https://mart.baemin.com/goods/detail/" + str(element['id'])

        datas.append({
            "category": categories[category],
            "title" : title,
            "price" : price,
#             "o_price" : o_price,
            "min_gram" : min_gram,
            "price_per_gram" : price_per_gram,
            "link" : link,
        })
        
    return pd.DataFrame(datas)

dfs = []
for category in categories:
    print(categories[category])
    for page in range(12): # url에서 page가 0부터 시작
        print(page, end=" ")
        df = baemin_items(category, page)
        dfs.append(df)
    print()
    time.sleep(2)

baemin_mart_delivery_goods_df = pd.concat(dfs, ignore_index=True)
baemin_mart_delivery_goods_df

engine = create_engine("mysql://{}:{}@{}/test?charset=utf8"
                       .format(sql_datas["user"], sql_datas["pw"], sql_datas["public_ip"]), encoding='utf8')
baemin_mart_delivery_goods_df.to_sql(name="baemin_del_goods", con=engine, if_exists="replace")

# baemin_mart_delivery_goods_df.to_csv("baemin_mart_delivery_goods.csv", index=False, encoding="utf-8-sig")

# print("="*100)
# print("It's saved as baemin_mart_delivery_goods.csv file.")
# print("="*100)


Writing baemin_del_sqlalchemy.py


> 초반 데이터베이스 생성 시 latin으로 되어있어서 utf8로 변경해줬어야 되는 오류였다.

> ## 최종 파일 

In [10]:
# %load baemin_del_sqlalchemy.py

import pandas as pd
import requests 
import time
from sqlalchemy import *
import configparser

config = configparser.ConfigParser()
config.read("../db.ini")
sql_datas = config["sql"]

# 카테고리 수정 전(참고용)
# categories = {
#     100104 : "플라스틱용기",
#     100105 : "종이용기/박스",
#     100138 : "카페/음료용기",
#     100159 : "종이/비닐봉투",
#     100165 : "수저/위생용품",
#     100177 : "스티커/배너/포스터",
#     100184 : "포장재",
#     100188 : "기타가게용품",
# }

# 카테고리 수정 후 
categories = {
    100104 : "포장용품",
    100105 : "포장용품",
    100138 : "포장용품",
    100159 : "포장용품",
    100165 : "가게위생용품",
    100184 : "포장용품",
    100188 : "가게위생용품",
}


def baemin_items(category, page):
    url = "https://mart.baemin.com/api/v2/front/categories/goods/{}/paging?goodsSortType=BASIC&page={}".format(category, page)
    response = requests.get(url)
    elements = response.json()['data']['simpleGoodsDtoPage']['content']
    datas = []
    for element in elements:
        title = element['name']
        price = element['goodsPrice']
#         o_price = element['customerPrice']
        # 원인은 모르겠지만.. 100050/밀가루 쪽에서 에러남
        try :
            min_gram = element['sizeDesc']
        except:
            min_gram = "-"
        try:
            price_per_gram = element['priceDesc']
        except:
            price_per_gram = "-"
        link = "https://mart.baemin.com/goods/detail/" + str(element['id'])

        datas.append({
            "category": categories[category],
            "title" : title,
            "price" : price,
#             "o_price" : o_price,
            "min_gram" : min_gram,
            "price_per_gram" : price_per_gram,
            "link" : link,
        })
        
    return pd.DataFrame(datas)

dfs = []
for category in categories:
    print(categories[category])
    for page in range(12): # url에서 page가 0부터 시작
        print(page, end=" ")
        df = baemin_items(category, page)
        dfs.append(df)
    print()
    time.sleep(2)

baemin_mart_delivery_goods_df = pd.concat(dfs, ignore_index=True)
baemin_mart_delivery_goods_df

engine = create_engine("mysql://{}:{}@{}/test?charset=utf8"
                       .format(sql_datas["user"], sql_datas["pw"], sql_datas["public_ip"]), encoding='utf8')
baemin_mart_delivery_goods_df.to_sql(name="baemin_del_goods", con=engine, if_exists="replace")

# baemin_mart_delivery_goods_df.to_csv("baemin_mart_delivery_goods.csv", index=False, encoding="utf-8-sig")

# print("="*100)
# print("It's saved as baemin_mart_delivery_goods.csv file.")
# print("="*100)


포장용품
0 1 2 3 4 5 6 7 8 9 10 11 
포장용품
0 1 2 3 4 5 6 7 8 9 10 11 
포장용품
0 1 2 3 4 5 6 7 8 9 10 11 
포장용품
0 1 2 3 4 5 6 7 8 9 10 11 
가게위생용품
0 1 2 3 4 5 6 7 8 9 10 11 
포장용품
0 1 2 3 4 5 6 7 8 9 10 11 
가게위생용품
0 1 2 3 4 5 6 7 8 9 10 11 


In [11]:
baemin_mart_delivery_goods_df

,category,title,price,min_gram,price_per_gram,link
0,포장용품,탕용기(소),57000,225Øx85(mm),개당 285원 / 200ea,https://mart.baemin.com/goods/detail/585
1,포장용품,[소량] 탕용기(소),29700,225Øx85(mm),개당 297원 / 100ea,https://mart.baemin.com/goods/detail/1778
2,포장용품,친환경 탕용기(소),30300,225Øx80(mm),개당 303원 / 100ea,https://mart.baemin.com/goods/detail/7446
3,포장용품,친환경 탕용기(소/JH),53500,220Øx82(mm),개당 268원 / 200ea,https://mart.baemin.com/goods/detail/9285
4,포장용품,친환경 탕용기(중/JH),57000,220Øx103(mm),개당 285원 / 200ea,https://mart.baemin.com/goods/detail/9286
...,...,...,...,...,...,...
2390,가게위생용품,[든든]리오 락스 2L,1050,2L,"개당 1,050원 / 1ea",https://mart.baemin.com/goods/detail/8833
2391,가게위생용품,[든든]스카치브라이트 수세미(다목적용_청색) 2 개,1980,2ea,"개당 1,980원 / 1ea",https://mart.baemin.com/goods/detail/8828
2392,가게위생용품,[든든]크린업 부직포행주(삼색_38*38cm) 3 개,770,3ea,개당 770원 / 1ea,https://mart.baemin.com/goods/detail/8826
2393,가게위생용품,[든든]크린업 스폰지수세미(15*9*3cm) 2 개,1170,2ea,"개당 1,170원 / 1ea",https://mart.baemin.com/goods/detail/8823


### MongoDB

In [34]:
%%writefile baemin_del_mongodb.py

import pandas as pd
import requests 
import time
import pymongo
import configparser

config = configparser.ConfigParser()
config.read("../db.ini")
mongodb = config["mongodb"]

categories = {
    100104 : "포장용품",
    100105 : "포장용품",
    100138 : "포장용품",
    100159 : "포장용품",
    100165 : "가게위생용품",
    100184 : "포장용품",
    100188 : "가게위생용품",
}


def baemin_items(category, page):
    url = "https://mart.baemin.com/api/v2/front/categories/goods/{}/paging?goodsSortType=BASIC&page={}".format(category, page)
    response = requests.get(url)
    elements = response.json()['data']['simpleGoodsDtoPage']['content']
    datas = []
    for element in elements:
        title = element['name']
        price = element['goodsPrice']
#         o_price = element['customerPrice']
        # 원인은 모르겠지만.. 100050/밀가루 쪽에서 에러남
        try :
            min_gram = element['sizeDesc']
        except:
            min_gram = "-"
        try:
            price_per_gram = element['priceDesc']
        except:
            price_per_gram = "-"
        link = "https://mart.baemin.com/goods/detail/" + str(element['id'])

        datas.append({
            "category": categories[category],
            "title" : title,
            "price" : price,
#             "o_price" : o_price,
            "min_gram" : min_gram,
            "price_per_gram" : price_per_gram,
            "link" : link,
        })
        
    return pd.DataFrame(datas)

dfs = []
for category in categories:
    print(categories[category])
    for page in range(12): # url에서 page가 0부터 시작
        print(page, end=" ")
        df = baemin_items(category, page)
        dfs.append(df)
    print()
    time.sleep(2)

baemin_mart_delivery_goods_df = pd.concat(dfs, ignore_index=True)
baemin_mart_delivery_goods_df

client = pymongo.MongoClient("mongodb://{}:{}@{}:27017/".format(mongodb["user"], mongodb["pw"], mongodb["public_ip"]))
baemin_del = client.baemin_db.baemin_del
items = baemin_mart_delivery_goods_df.to_dict('records')
baemin_del.insert(items)

# baemin_mart_delivery_goods_df.to_csv("baemin_mart_delivery_goods.csv", index=False, encoding="utf-8-sig")

print("="*60)
print("baemin data is in MongoDB..")
print("="*60)


Overwriting baemin_del_mongodb.py


In [32]:
client = pymongo.MongoClient("mongodb://{}:{}@{}:27017/".format(mongodb["user"], mongodb["pw"], mongodb["public_ip"]))
baemin_del = client.baemin_db.baemin_del
items = baemin_mart_delivery_goods_df.to_dict('records')
log_data = baemin_del.insert(items)
len(log_data)

2395

In [ ]:
# %load baemin_del_mongodb.py

import pandas as pd
import requests 
import time
import pymongo
import configparser

config = configparser.ConfigParser()
config.read("../db.ini")
mongodb = config["mongodb"]

categories = {
    100104 : "포장용품",
    100105 : "포장용품",
    100138 : "포장용품",
    100159 : "포장용품",
    100165 : "가게위생용품",
    100184 : "포장용품",
    100188 : "가게위생용품",
}


def baemin_items(category, page):
    url = "https://mart.baemin.com/api/v2/front/categories/goods/{}/paging?goodsSortType=BASIC&page={}".format(category, page)
    response = requests.get(url)
    elements = response.json()['data']['simpleGoodsDtoPage']['content']
    datas = []
    for element in elements:
        title = element['name']
        price = element['goodsPrice']
#         o_price = element['customerPrice']
        # 원인은 모르겠지만.. 100050/밀가루 쪽에서 에러남
        try :
            min_gram = element['sizeDesc']
        except:
            min_gram = "-"
        try:
            price_per_gram = element['priceDesc']
        except:
            price_per_gram = "-"
        link = "https://mart.baemin.com/goods/detail/" + str(element['id'])

        datas.append({
            "category": categories[category],
            "title" : title,
            "price" : price,
#             "o_price" : o_price,
            "min_gram" : min_gram,
            "price_per_gram" : price_per_gram,
            "link" : link,
        })
        
    return pd.DataFrame(datas)

dfs = []
for category in categories:
    print(categories[category])
    for page in range(12): # url에서 page가 0부터 시작
        print(page, end=" ")
        df = baemin_items(category, page)
        dfs.append(df)
    print()
    time.sleep(2)

baemin_mart_delivery_goods_df = pd.concat(dfs, ignore_index=True)
baemin_mart_delivery_goods_df

client = pymongo.MongoClient("mongodb://{}:{}@{}:27017/".format(mongodb["user"], mongodb["pw"], mongodb["public_ip"]))
baemin_del = client.baemin_db.baemin_del
items = baemin_mart_delivery_goods_df.to_dict('records')
baemin_del.insert(items)

# baemin_mart_delivery_goods_df.to_csv("baemin_mart_delivery_goods.csv", index=False, encoding="utf-8-sig")

print("="*60)
print("baemin data is in MongoDB..")
print("="*60)
